In [60]:
import secrets
import hashlib
from Crypto.Cipher import ChaCha20_Poly1305, ChaCha20
from Crypto.Hash import Poly1305
import struct
from Crypto.Util.number import bytes_to_long, inverse, long_to_bytes

# flag = open("flag.txt").read().strip()
flag = "What is your name?"

def shasum(x):
    return hashlib.sha256(x).digest()

key = shasum(shasum(secrets.token_bytes(32) + flag.encode()))

# Generate a random nonce to be extra safe
nonce = secrets.token_bytes(12)

messages = [
    "Did you know that ChaCha20-Poly1305 is an authenticated encryption algorithm?",
    "That means it protects both the confidentiality and integrity of data!",
    "But it's only secure if used correctly! fsaj kf hjas hfjaf hajf asfafj a fjas"
]

goal = "But it's only secure if used correctly!"

def encrypt(message):
    cipher = ChaCha20_Poly1305.new(key=key, nonce=nonce)
    ciphertext, tag = cipher.encrypt_and_digest(message)
    return ciphertext + tag + nonce

def decrypt(message_enc):
    ciphertext = message_enc[:-28]
    tag = message_enc[-28:-12]
    nonce = message_enc[-12:]
    cipher = ChaCha20_Poly1305.new(key=key, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)
    return plaintext

In [78]:
def compute_poly1305_tag(poly1305_key, ciphertext, aad=b'\x00', nonce = nonce):
    """Tính toán lại tag Poly1305"""
    # self._len_ct += len(plaintext) = len(plaintext)
    # self._authenticator.update(result)
    # 
    # qua phương thức _compute_mac():
    # if self._len_ct & 0x0F:
    #    self._authenticator.update(b'\x00' * (16 - (self._len_ct & 0x0F)))

    # self._authenticator.update(long_to_bytes(self._len_aad, 8)[::-1])
    # self._authenticator.update(long_to_bytes(self._len_ct, 8)[::-1])
    # self._mac_tag = self._authenticator.digest()
    # return self._mac_tag
    
    poly1305_mac = Poly1305.new(key=poly1305_key, cipher=ChaCha20, nonce=nonce)
    poly1305_mac.update(ciphertext)                                     # update lần 1
    
    if len(ciphertext) % 16 != 0:
        poly1305_mac.update(aad * (16 - (len(ciphertext) % 16)))        # update lần 2
    
    poly1305_mac.update(long_to_bytes(0, 8)[::-1])                      # update lần 3
    poly1305_mac.update(long_to_bytes(len(ciphertext), 8)[::-1])        # update lần 4
    return poly1305_mac.digest()

mes1 = messages[0].encode()
mes2 = messages[1].encode()

enc1 = encrypt(mes1)
enc2 = encrypt(mes2)

cipher1 = enc1[:-28]  
tag1 = enc1[-28:-12]
nonce1 = enc1[-12:]

cipher2 = enc2[:-28]
tag2 = enc2[-28:-12]
nonce2 = enc2[-12:]

assert nonce1 == nonce2

keystream1 = bytes([a ^ b for a, b in zip(cipher1, mes1)])
keystream2 = bytes([a ^ b for a, b in zip(cipher2, mes2)])

mes3 = messages[2].encode()
cipher3 = bytes([a ^ b for a, b in zip(mes3, keystream1)])
tag3 = compute_poly1305_tag(key[:32], cipher3, nonce=nonce1)
res = cipher1 + tag1 + nonce1   

print(len(res), res.hex())
print(len(enc1), enc1.hex())

user = bytes.fromhex(res.hex())
user_message = decrypt(user)
print("User message (decrypted): " + repr(user_message))

105 57c16057a4801a9f29f36807ca82434a0799a60b5bcf916a6011c44fedccaa706fa8e54f622d7139510e9f2eaf1d83908ba29e3235a18964bc8d88ae2851ed3bf848fb45508eec002379204a75c9fe63147febbed967a91170b302f84960a8a98b8a5197daebb73574
105 57c16057a4801a9f29f36807ca82434a0799a60b5bcf916a6011c44fedccaa706fa8e54f622d7139510e9f2eaf1d83908ba29e3235a18964bc8d88ae2851ed3bf848fb45508eec002379204a75c9fe63147febbed967a91170b302f84960a8a98b8a5197daebb73574
User message (decrypted): b'Did you know that ChaCha20-Poly1305 is an authenticated encryption algorithm?'
